In [1]:
import json
import numpy as np
from IPython.display import display, HTML
import pandas as pd
pd.set_option("display.precision", 3)

In [2]:
planner_names = [
    "MTGTaskPlanner",
    "MTGTaskPlannerWithGMM",
    # "DynamicMTGTaskPlannerWithGMM_25.0",
    # "DynamicMTGTaskPlannerWithGMM_50.0",
    # "DynamicMTGTaskPlannerWithGMM_75.0",
    # "DynamicMTGTaskPlannerWithGMM_100.0",
    "MTGTaskPlannerWithAtlas",
    # "DynamicMTGTaskPlannerWithAtlas_25.0",
    # "DynamicMTGTaskPlannerWithAtlas_50.0",
    # "DynamicMTGTaskPlannerWithAtlas_75.0",
    # "DynamicMTGTaskPlannerWithAtlas_100.0",

]


In [3]:
def read_json(file_path):
    """
    Reads a json file
    """
    result_data = []
    # Open the file and load JSON data
    with open(file_path, 'r') as json_file:
        for line in json_file:
            try:
                result_data.append(json.loads(line))
            except:
                continue
    return result_data
    

In [6]:
filepaths = ["../result.json"]


for file_path in filepaths:
    print(file_path)
    result_data = read_json(file_path)
    planner_stats = {
                        planner_name: {
                            'num' : 0, 
                            'num_success' : [], 
                            'motion_planning_time' : [],
                            'task_planning_time' : [],
                            'updating_time' : [],
                            'solution_length' : [],
                            'num_attempts' : [],
                            'total_planning_time' : [],
                            'problem_load_time' : [],
                            'set_start_and_goal_time' : [],
                        } 

                     for planner_name in planner_names
                    }

    for d in result_data:
        if d['planner_name'] in planner_stats:
            dic = planner_stats[d['planner_name']]
            dic['num'] += 1
            dic['num_success'].append(d['success'] == "true")
            if d['success'] == "true":
                dic['motion_planning_time'].append(d['motion_planning_time'])
                dic['task_planning_time'].append(d['task_planning_time'])
                dic['total_planning_time'].append(d['total_planning_time'])
                dic['updating_time'].append(d['updating_time'])
                dic['solution_length'].append(d['solution_length'])
                dic['num_attempts'].append(d['num_attempts'])
                dic['problem_load_time'].append(d['problem_load_time'])
                dic['set_start_and_goal_time'].append(d['set_start_and_goal_time'])

    output_dict = []
    for planner_name in planner_names:
        stats = planner_stats[planner_name]

        current_planner_eval_dict = {}
        current_planner_eval_dict["Planner Name"] = planner_name
        current_planner_eval_dict["Total Number of Tasks"] = len(stats['num_success'])
        current_planner_eval_dict["Success Rate"] = np.mean(stats['num_success']) * 100.0
        current_planner_eval_dict["Mean Task Planning Time"] = np.mean(np.array(stats['task_planning_time']) / (np.array(stats['num_attempts'])) )
        current_planner_eval_dict["Mean Motion Planning Time"] = np.mean(np.array(stats['motion_planning_time']) / (np.array(stats['num_attempts'])) )
        current_planner_eval_dict["Total Planning Time"] = np.mean(np.array(stats['total_planning_time']) )
        current_planner_eval_dict["Problem Load Time"] = np.mean(np.array(stats['problem_load_time']) )
        current_planner_eval_dict["Set Start and Goal Time"] = np.mean(np.array(stats['set_start_and_goal_time']) )
        current_planner_eval_dict["Mean Update time"] = np.mean(np.array(stats['updating_time']) / (np.array(stats['num_attempts'])) )
        current_planner_eval_dict["Mean solution length"] = np.mean(np.array(stats['solution_length']) / (np.array(stats['num_attempts'])) )
        current_planner_eval_dict["Mean number of attempts"] = np.mean((np.array(stats['num_attempts'])) )
        output_dict.append(current_planner_eval_dict)
    df = pd.DataFrame(output_dict).set_index('Planner Name')

    display(df)

../result.json


/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python2.7/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,Mean Motion Planning Time,Mean Task Planning Time,Mean Update time,Mean number of attempts,Mean solution length,Problem Load Time,Set Start and Goal Time,Success Rate,Total Number of Tasks,Total Planning Time
Planner Name,,,,,,,,,,
MTGTaskPlanner,0.568,0.004,7.868e-06,1.0,29.622,0.067,1.659e-04,100.0,1,0.572
MTGTaskPlannerWithGMM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
MTGTaskPlannerWithAtlas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
